<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/AnuvaadVsSelf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install scipy

In [ ]:
!pip install sentence-splitter

In [ ]:
!pip install indic-nlp-library

In [28]:
import sys
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import sentence_tokenize 
from sentence_transformers import SentenceTransformer
import numpy as np
import re 
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from scipy.spatial import distance

In [6]:
model = SentenceTransformer('LaBSE')

100%|██████████| 1.75G/1.75G [01:05<00:00, 26.8MB/s]


In [7]:
def anuvaad_aligner(src, tgtp):
      '''
      This function is meant to align src phrases with best possible tgt phrase using LABSE model
      '''
      out = {}
      aligned_phrases = {}
      print("Performing phrase alignenment using LABSE")
      src_phrases, tgt = src, tgtp
      
      for src_phrase in src_phrases:
          length_src_phrase = len(src_phrase.split())           
          tgt_token_list = split_tgt(length_src_phrase,tgt)
          embeddings_src_phrase, embeddings_tgt_tokens = generate_embeddings([src_phrase],tgt_token_list)
          alignments = get_target_sentence(embeddings_tgt_tokens, embeddings_src_phrase, length_src_phrase)
      
          if alignments is not None and alignments[2] is "MATCH":
              aligned_phrases[src_phrase] = tgt_token_list[alignments[0]]
          elif alignments is not None and alignments[2] is "NOMATCH": 
              print("No exact match found for:{} . Possible alignment {}".format(src_phrase,tgt_token_list[alignments[0]]))  
                      
      print("Aligned Phrases: {}".format(aligned_phrases))
      out = {"tgt":tgt,"src_phrases":src_phrases,"aligned_phrases":aligned_phrases}     

      return out

def split_tgt(length_src_phrase,tgt):
  tgt_token_list = list()
  tokenised_tgt = tgt.split()
  tgt_token_list = [tokenised_tgt[i:i+length_src_phrase] for i in range(len(tokenised_tgt)) if (i + length_src_phrase) <= len(tokenised_tgt)]
  tgt_token_list_plus = [tokenised_tgt[i:i+length_src_phrase+1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase+1) <= len(tokenised_tgt)]
  tgt_token_list_minus = [tokenised_tgt[i:i+length_src_phrase-1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase-1) <= len(tokenised_tgt) and 
                          length_src_phrase != 1]
  tgt_token_list = tgt_token_list + tgt_token_list_plus + tgt_token_list_minus
  tgt_token_list = [" ".join(j) for j in tgt_token_list]
  return tgt_token_list
      
def generate_embeddings(input_1, input_2):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(input_1,show_progress_bar=True)
  embeddings_input_2 = model.encode(input_2,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  
def get_target_sentence(target_embeddings, source_embedding, length_src_phrase):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  print("Match score: {}".format(min_distance))
  if min_distance >= 0.5:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     
      

In [43]:
content = '''Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February

Chandigarh, 12 February:

Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.

Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter. Taking immediate action on this, Haryana State Women's Commission, Mr. K.K. A letter was written to Selvaraja instructing that according to Section 3 (10), (1) (f) (k) of the Haryana State Women's Commission Act, ensure legal aid to Naudip Kaur, a resident of village Giadad in Muktsar district of Punjab state. Along with making, the security of the hawatis should also be made sure.'''
content = content.replace('\n',' ')
splitter = SentenceSplitter(language='en')
so_sentences = splitter.split(content)
tgtp = '''करनाल जेल में बंद नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात

चंडीगढ़, 12 फरवरी:

करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीषा गुलाटी 15 फऱवरी, 2021 को दोपहर 12 बजे मुलाकात करेंगी। 

इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी। इस पर तुरंत कार्यवाही करते हुए हरियाणा राज्य महिला आयोग द्वारा हरियाणा के जेल विभाग के डायरैक्टर जनरल श्री के. सेल्वाराज को पत्र लिखकर हिदायत की गई कि हरियाणा राज्य महिला आयोग एक्ट की धारा 3(10), (1)(एफ)(के) के अनुसार पंजाब राज्य के मुक्तसर जि़ले के गाँव गियादड़ की निवासी नौदीप कौर को कानूनी सहायता मुहैया करवाने को सुनिश्चित बनाने के साथ-साथ हवालाती की सुरक्षा को भी यकीनी बनाया जाए। 
'''
src = so_sentences 
res = anuvaad_aligner(src, tgtp)

Performing phrase alignenment using LABSE



LABSE embedding generation finished
Match score: 0.9515082939906941



LABSE embedding generation finished
Match score: 0.9125870724667297



LABSE embedding generation finished
Match score: 0.8503684323609959



LABSE embedding generation finished
Match score: 0.7746197979584467
Aligned Phrases: {"Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.": 'नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात चंडीगढ़, 12 फरवरी: करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीषा गुलाटी 15 फऱवरी, 2021 को दोपहर 12 बजे मुलाकात करेंगी। इस सम्बन्धी', "Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.": 'करेंगी। इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी।

In [44]:
for key in res['aligned_phrases']:
  print(key)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
  print(res['aligned_phrases'][key])  

Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.
नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात चंडीगढ़, 12 फरवरी: करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीषा गुलाटी 15 फऱवरी, 2021 को दोपहर 12 बजे मुलाकात करेंगी। इस सम्बन्धी
Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.
करेंगी। इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी। इस
Taking immediate action on this, Haryana State Women's Commission, Mr. K.K. A letter was writ

My Version

In [10]:
def LaBSEembeddings(source, target):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(source,show_progress_bar=True)
  embeddings_input_2 = model.encode(target,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  

In [50]:
def match_target_sentence(source_embedding, target_embeddings):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  if min_distance >= 0.3:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     

In [51]:
def self_aligner(source_embeddings, target_embeddings):
  target_map = {}
  source_map = {}
  for s in range(len(source_embeddings)):
    source_map[s] = so_sentences[s]
  for t in range(len(target_embeddings)):
    target_map[t] = ta_sentences[t]
  print(source_map, target_map)
  aligned_phrases= {}
  for s in range(len(source_embeddings)) :
      alignments = match_target_sentence([source_embeddings[s]], target_embeddings)
      if alignments is not None and alignments[2] is "MATCH":
        l = []
        l.append(target_map[alignments[0]])
        l.append(alignments[1])
        aligned_phrases[source_map[s]] = l
  return aligned_phrases   

In [54]:
content =  '''Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February
Chandigarh, 12 February:
Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.
Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter. Taking immediate action on this, Haryana State Women's Commission, Mr. K.K. A letter was written to Selvaraja instructing that according to Section 3 (10), (1) (f) (k) of the Haryana State Women's Commission Act, ensure legal aid to Naudip Kaur, a resident of village Giadad in Muktsar district of Punjab state. 
Along with making, the security of the hawatis should also be made sure.'''
content = content.replace('\n',' ')
splitter = SentenceSplitter(language='en')
so_sentences = splitter.split(content)
content = tgtp
content = content.replace('\n',' ')
ta_sentences = sentence_tokenize.sentence_split(content, lang='hi')
source_embeddings, target_embeddings = LaBSEembeddings(so_sentences, ta_sentences)
res_m = self_aligner(source_embeddings, target_embeddings)


LABSE embedding generation finished
{0: "Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.", 1: "Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.", 2: "Taking immediate action on this, Haryana State Women's Commission, Mr. K.K. A letter was written to Selvaraja instructing that according to Section 3 (10), (1) (f) (k) of the Haryana State Women's Commission Act, ensure legal aid to Naudip Kaur, a resident of village Giadad in Muktsar district of Punjab state.", 3: 'Along with making, the security of the hawatis should also be made sure.'} {0: 'करनाल जेल में बंद नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाका

In [53]:
res_m

{"Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.": ['इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी।',
  0.8991213259493398],
 "Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.": ['करनाल जेल में बंद नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात    चंडीगढ़, 12 फरवरी:    करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीषा गुलाटी 15 फऱवरी, 2021 को दोपहर 12 बजे मुलाकात करेंगी।',
  0.9571049857044178],
 "Taking immediate action on t

In [45]:
res_m

{"Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.": ['इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी।',
  0.8991213259493398],
 "Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.": ['करनाल जेल में बंद नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात    चंडीगढ़, 12 फरवरी:    करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीषा गुलाटी 15 फऱवरी, 2021 को दोपहर 12 बजे मुलाकात करेंगी।',
  0.9571049857044178],
 "Taking immediate action on t

In [47]:
res['aligned_phrases']

{'Along with making, the security of the hawatis should also be made sure.': 'सुनिश्चित बनाने के साथ-साथ हवालाती की सुरक्षा को भी यकीनी बनाया जाए।',
 "Giving this information, the spokesperson of the commission said that the matter was raised by the chairperson of the Punjab State Women's Commission before the counterpart of the state of Haryana and appealed to them to intervene in the matter.": 'करेंगी। इस सम्बन्धी जानकारी देते हुए आयोग के प्रवक्ता ने बताया कि पंजाब राज्य महिला आयोग की चेयरपर्सन द्वारा यह मामला हरियाणा राज्य की समकक्ष के समक्ष उठाया था और उनको इस मामले में दख़ल देने की अपील की थी। इस',
 "Manisha Gulati to meet Naudip Kaur in Karnal jail on 15th February Chandigarh, 12 February: Punjab State Women's Commission chairperson Smt Manisha Gulati will meet Naudip Kaur, who is in Karnal jail, on 15 February 2021 at 12 noon.": 'नौदीप कौर से मनीषा गुलाटी करेंगी 15 फऱवरी को मुलाकात चंडीगढ़, 12 फरवरी: करनाल जेल में बंद नौदीप कौर से पंजाब राज्य महिला आयोग की चेयरपर्सन श्रीमती मनीष